In [127]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

population_00 = pd.read_csv("../Raochuan/nyc_tract_2000.csv")
population_00.drop('Total Population',1,inplace = True)
population_00.drop('Median Age',1,inplace = True)
population_00.columns.values[1] = '2010 Census Tract'
population_00.head()
population_01 = pd.read_csv("../Haichao/2001_predict_backward.csv")
population_02 = pd.read_csv("../Haichao/2002_predict_backward.csv")
population_03 = pd.read_csv("../Haichao/2003_predict_backward.csv")
population_04 = pd.read_csv("../Haichao/2004_predict_backward.csv")
population_05 = pd.read_csv("../Haichao/2005_predict_backward.csv")
population_06 = pd.read_csv("../Haichao/2006_predict_backward.csv")
population_07 = pd.read_csv("../Haichao/2007_predict_backward.csv")
population_08 = pd.read_csv("../Haichao/2008_predict_backward.csv")
population_09 = pd.read_csv("../Haichao/2009_predict_backward.csv",names = population_08.columns.values[2:])
population_09.insert(0,'Borough',population_08['Borough'])
population_09.insert(1,'2010 Census Tract',population_08['2010 Census Tract'])
population_10 = pd.read_csv("nyc_tract_2010.csv")
population_10.drop('Total Population',1,inplace = True)
population_10.drop('Median Age',1,inplace = True)

In [135]:
# The 2000 data is special, so we handle it indepedently
ct_index = raw_data['index']
population_00 = population_00.ix[population_00['Borough'] == 'Brooklyn']
population_00
ct_index[ct_index % 1 != 0]=ct_index*100
ct_index = pd.DataFrame(data = ct_index)

data_2000 = pd.merge(ct_index,population_00,left_on='index',right_on='2010 Census Tract',how='left')
data_2000 = data_2000[['index', 'Under 5 Years', '5-9      Years', '10-14 Years']]
for i in range(len(data_2000['index'])):
    if (data_2000.iloc[i]['index'] > 1144):
        data_2000.set_value(data_2000.index.values[i],'index', data_2000.iloc[i]['index']*0.01)
data_2000.columns = ['2010 Census Tract', 'Under 5 Years', '5-9      Years', '10-14 Years']
data_2000.fillna(0, inplace = True)

,2010 Census Tract,Under 5 Years,5-9 Years,10-14 Years
0,18.00,5,1,0
1,20.00,142,137,108
2,22.00,307,298,273
3,30.00,130,116,102
4,34.00,167,137,123
5,36.00,186,163,181
6,38.00,85,76,81
7,44.00,0,0,0
8,46.00,65,86,92
9,50.00,133,91,110


In [136]:
def filter_brooklyn_20(yearly_data):
    yearly_data = yearly_data.ix[yearly_data['Borough'] == 'Brooklyn']
    yearly_data['2010 Census Tract'] = [x / 100 for x in yearly_data['2010 Census Tract']]
    raw_data = pd.read_csv("../Raochuan/Kth_09_10.csv")
    ct_index = raw_data['index']
    yearly_data = yearly_data.ix[yearly_data['2010 Census Tract'].isin(ct_index)]
    yearly_data = yearly_data[['2010 Census Tract', 'Under 5 Years', '5-9      Years', '10-14 Years']]
    return yearly_data

In [137]:
data_2001 = filter_brooklyn_20(population_01)
data_2002 = filter_brooklyn_20(population_02)
data_2003 = filter_brooklyn_20(population_03)
data_2004 = filter_brooklyn_20(population_04)
data_2005 = filter_brooklyn_20(population_05)
data_2006 = filter_brooklyn_20(population_06)
data_2007 = filter_brooklyn_20(population_07)
data_2008 = filter_brooklyn_20(population_08)
data_2009 = filter_brooklyn_20(population_09)
data_2010 = filter_brooklyn_20(population_10)

In [138]:
def diff(former_data, latter_data,year):
    diff = pd.DataFrame(columns=['2010 Census Tract', year+'_0-4', year+'_5-9', year+'_10-14'])
    diff['2010 Census Tract'] = former_data['2010 Census Tract']
    diff[year+'_0-4'] = [a - b for a, b in zip(latter_data['Under 5 Years'], former_data['Under 5 Years'])]
    diff[year+'_5-9'] = [a - b for a, b in zip(latter_data['5-9      Years'], former_data['5-9      Years'])]
    diff[year+'_10-14'] = [a - b for a, b in zip(latter_data['10-14 Years'], former_data['10-14 Years'])]
    return diff

In [147]:
diff20012002 = diff(data_2001, data_2002, "20012002")
diff20022003 = diff(data_2002, data_2003, "20022003")
diff20032004 = diff(data_2003, data_2004, "20032004")
diff20042005 = diff(data_2004, data_2005, "20042005")
diff20052006 = diff(data_2005, data_2006, "20052006")
diff20062007 = diff(data_2006, data_2007, "20062007")
diff20072008 = diff(data_2007, data_2008, "20072008")
diff20082009 = diff(data_2008, data_2009, "20082009")
diff20092010 = diff(data_2009, data_2010, "20092010")

diff20012002.head()

,2010 Census Tract,20012002_0-4,20012002_5-9,20012002_10-14
349,18.0,0,0,0
350,20.0,3,-4,2
352,22.0,7,-8,2
355,30.0,2,-3,2
358,34.0,5,-6,1


In [156]:
raw_data = pd.read_csv("../Raochuan/Kth_09_10.csv")
data = raw_data[['index']]
data.columns=['2010 Census Tract']
data['Wht%'] = raw_data['White'] / raw_data['total']
data['Blc%'] = raw_data.iloc[:,6] / raw_data['total']
data['Ind%'] = raw_data.iloc[:,7] / raw_data['total']
data['Asn%'] = raw_data.iloc[:,8] / raw_data['total']
data['Mix%'] = raw_data.iloc[:,11] / raw_data['total']
data['His%'] = raw_data.iloc[:,12] / raw_data['total']

data.head()


,2010 Census Tract,Wht%,Blc%,Ind%,Asn%,Mix%,His%
0,18.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667
1,20.0,0.021008,0.012605,0.006303,0.042017,0.006303,0.909664
2,22.0,0.069246,0.029532,0.004073,0.082485,0.020367,0.787169
3,30.0,0.535912,0.024862,0.000000,0.129834,0.082873,0.226519
4,34.0,0.559415,0.000000,0.000000,0.186472,0.047532,0.206581


In [154]:
def insert_data(integrated_data, insert_data):
    integrated_data[insert_data.columns[1]] = insert_data.iloc[:,1]
    integrated_data[insert_data.columns[2]] = insert_data.iloc[:,2]
    integrated_data[insert_data.columns[3]] = insert_data.iloc[:,3]
    print(integrated_data)
    return integrated_data

In [162]:
new = pd.merge(data,diff20012002,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20022003,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20032004,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20042005,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20052006,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20062007,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20072008,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20082009,how='left',on='2010 Census Tract')
new = pd.merge(new,diff20092010,how='left',on='2010 Census Tract')
new.to_csv("CTdata.csv")

In [163]:
new.head()

,2010 Census Tract,Wht%,Blc%,Ind%,Asn%,Mix%,His%,20012002_0-4,20012002_5-9,20012002_10-14,...,20062007_10-14,20072008_0-4,20072008_5-9,20072008_10-14,20082009_0-4,20082009_5-9,20082009_10-14,20092010_0-4,20092010_5-9,20092010_10-14
0,18.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667,0,0,0,...,0,0,0,0,1.0,0.0,0.0,-1.0,0.0,0.0
1,20.0,0.021008,0.012605,0.006303,0.042017,0.006303,0.909664,3,-4,2,...,-4,3,6,-2,0.0,2.0,-6.0,-16.0,-15.0,-2.0
2,22.0,0.069246,0.029532,0.004073,0.082485,0.020367,0.787169,7,-8,2,...,-9,6,13,-4,3.0,2.0,-12.0,-38.0,-32.0,-4.0
3,30.0,0.535912,0.024862,0.000000,0.129834,0.082873,0.226519,2,-3,2,...,-4,2,5,-2,1.0,1.0,-4.0,-13.0,-10.0,-2.0
4,34.0,0.559415,0.000000,0.000000,0.186472,0.047532,0.206581,5,-6,1,...,-6,4,6,-2,2.0,3.0,-6.0,-22.0,-17.0,-2.0
